In [6]:
import pandas as pd
import pandas_profiling
import numpy as np
import json
from jinja2 import Environment, FileSystemLoader

%%javascript
require.config({
paths: {
d3: "https://d3js.org/d3.v5.min",
}
});

require(["d3"], function(d3) {
window.d3 = d3;
});

def build_data(df):
    count = []
    considered_cols = []
    last_index = 0
    keys = {}

    df.replace(to_replace=["na", "?", np.nan, "missing", "not available",
                           "n/a", "missing value"], value=np.nan, inplace=True)

    for col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='ignore')

    for col in df.columns:
        if df[col].dtype == "object":
            df[col].replace(to_replace=["na", "?", np.nan, "missing", "not available",
                                        "n/a", "missing value"], value="missing value", inplace=True)
            if df[col].unique().size < 25:
                considered_cols.append(col)
                keys[col] = {"values": df[col].unique().tolist(
                ), "start_i": last_index, "end_i": last_index + df[col].unique().size}
                last_index = last_index + df[col].unique().size
                for item in df[col].unique():
                    count.append([col, item])
    data_array = np.zeros((len(count), len(count)))

    for row in range(df.shape[0]):
        for col in range(df.shape[1]):
            for i in range(col+1, df.shape[1]):
                if df.columns[col] in considered_cols and df.columns[i] in considered_cols:
                    data_array[count.index([df.columns[col], df.iloc[row, col]]), count.index(
                        [df.columns[i], df.iloc[row, i]])] += 1

    data_array += np.transpose(data_array)
    return keys, data_array.tolist()

file_loader = FileSystemLoader("templates")
env = Environment(loader=file_loader)

template = env.get_template("bid.html")

keys, data_array = build_data(df)

output = template.render(data=keys, full_matrix=data_array)

with open("bid_out.html", "w") as result_file:
    result_file.write(output)

In [ ]:
%%javascript
require.config({
paths: {
d3: "https://d3js.org/d3.v5.min",
}
});

require(["d3"], function(d3) {
window.d3 = d3;
});

In [ ]:
import pandas as pd
import pandas_profiling
import numpy as np

df = pd.read_csv('<arquivo>.csv')

pandas_profiling.ProfileReport(df)
